Cafe search

Business description: My client is a bussinesman that would like to open his first cafe. He does not know about the business and would like to visit cafes to benchamark and gain new business ideas. The problem is that he has a limited budget and he can visit only one city. His business idea is inspired in an European style cafe, so he has in mind Paris, London, and Vienna as possible cities. 
He is asking me which of this 3 cities he should visit, this means: which is the one that has a larger ammounts of cafes in a certain area.
Just to simplify the problem, we will concentrate in an area of radius 2000 meters.

Data description

My two data sources will be google maps and foursquare.
From google maps I will obtain the latitude and longitude of the city centers of the three cities that I want to study.
Then, with my Foursquare credentials I will perform 3 searches for a specific venue category filtering by "Cafe", one search for each city.
The result will be stored as a dataframe and processed. I will have 3 dataframes.
Finally I will count the ammount of cafes that each city has, and give a recommendation.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 112kB 2.9MB/s eta 0:00:01
Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'NUFA0BDTFEQO5Z4MLXECORFESJLZYUCPFOXKHYL2XAG0B1Z1' # your Foursquare ID
CLIENT_SECRET = '5PGQOVGK4W2SUJDCEIJBYUAYYD3USGVU1GUKCQABOZVDHCU1' # your Foursquare Secret
ACCESS_TOKEN = 'FARG2Q33MHNYLA53A3YUQQ4CDGZGRQURIJRV5E1C02RYIS04' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NUFA0BDTFEQO5Z4MLXECORFESJLZYUCPFOXKHYL2XAG0B1Z1
CLIENT_SECRET:5PGQOVGK4W2SUJDCEIJBYUAYYD3USGVU1GUKCQABOZVDHCU1


In [4]:
address = 'London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
london_latitude = location.latitude
london_longitude = location.longitude

address = 'Paris'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
paris_latitude = location.latitude
paris_longitude = location.longitude

address = 'Vienna'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
vienna_latitude = location.latitude
vienna_longitude = location.longitude
print(london_latitude, london_longitude)
print(paris_latitude, paris_longitude)
print(vienna_latitude, vienna_longitude)

51.5073219 -0.1276474
48.8566969 2.3514616
48.2083537 16.3725042


In [5]:
search_query = 'Cafe'
radius = 2000
print(search_query + ' .... OK!')

Cafe .... OK!


In [7]:
london_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, london_latitude, london_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
london_url

paris_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, paris_latitude, paris_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
paris_url

vienna_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, vienna_latitude, vienna_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
vienna_url

'https://api.foursquare.com/v2/venues/search?client_id=NUFA0BDTFEQO5Z4MLXECORFESJLZYUCPFOXKHYL2XAG0B1Z1&client_secret=5PGQOVGK4W2SUJDCEIJBYUAYYD3USGVU1GUKCQABOZVDHCU1&ll=48.2083537,16.3725042&oauth_token=FARG2Q33MHNYLA53A3YUQQ4CDGZGRQURIJRV5E1C02RYIS04&v=20180604&query=Cafe&radius=2000&limit=30'

In [9]:
london_results = requests.get(london_url).json()
paris_results = requests.get(paris_url).json()
vienna_results = requests.get(vienna_url).json()
london_results
paris_results
vienna_results

{'meta': {'code': 200, 'requestId': '600c29bf148f584621646644'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4b963f8df964a52051c234e3',
    'name': "Café de l'Europe",
    'location': {'address': 'Graben 31',
     'lat': 48.2082251445176,
     'lng': 16.37112147999093,
     'labeledLatLngs': [{'label': 'display',
       'lat': 48.2082251445176,
       'lng': 16.37112147999093}],
     'distance': 103,
     'postalCode': '1010',
     'cc': 'AT',
     'city': 'Wien',
     'state': 'Wien',
     'country': 'Österreich',
     'formattedAddress': ['Graben 31', '1010 Wien', 'Österreich']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1611409855',
    'hasPerk': False},
   {'id': '4c25ebb2f7ced13

In [10]:
# assign relevant part of JSON to venues
london_venues = london_results['response']['venues']

# tranform venues into a dataframe
london_dataframe = json_normalize(london_venues)
london_dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4b228bfef964a520c34824e3,Café In The Crypt,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,St. Martin-in-the-Fields,51.508962,-0.126729,"[{'label': 'display', 'lat': 51.50896230356763...",193,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin-in-the-Fields, London, Greater Lon...",NaN,NaN,NaN
1,4af4407cf964a520e3f021e3,Cafe On The Square,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,Trafalgar Sq,51.508304,-0.127786,"[{'label': 'display', 'lat': 51.50830388767846...",109,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,NaN
2,4ba3cdf5f964a520606138e3,The National Café,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,Trafalgar Sq,51.508803,-0.127544,"[{'label': 'display', 'lat': 51.50880255694977...",164,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq (St. Martin's Pl), London, Great...",St. Martin's Pl,NaN,NaN
3,4db549dccda1c57c826ad233,Café Fiori,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1611409855,False,43 Cranbourn St,51.511280,-0.128596,"[{'label': 'display', 'lat': 51.51128033777337...",445,WC2H 7AN,GB,London,Greater London,United Kingdom,"[43 Cranbourn St, London, Greater London, WC2H...",NaN,NaN,NaN
4,55a7786b498ecd02e85c3b26,Portrait Café,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,NaN,51.509324,-0.127369,"[{'label': 'display', 'lat': 51.50932448760624...",223,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,NaN


In [11]:
# assign relevant part of JSON to venues
paris_venues = paris_results['response']['venues']

# tranform venues into a dataframe
paris_dataframe = json_normalize(paris_venues)
paris_dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,4afac61bf964a5209c1822e3,Café L’Étincelle,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,42 bis rue de Rivoli,48.856623,2.355777,"[{'label': 'display', 'lat': 48.85662349708642...",316,75004,FR,Paris,Île-de-France,France,"[42 bis rue de Rivoli, 75004 Paris, France]",NaN,NaN
1,51c5c23d498e3a8ed20914ba,Café Clémentine,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,4 rue de la Coutellerie,48.857211,2.350346,"[{'label': 'display', 'lat': 48.85721121243165...",99,75004,FR,Paris,Île-de-France,France,"[4 rue de la Coutellerie, 75004 Paris, France]",NaN,NaN
2,4bb5dd68941ad13a6c941ee3,Café Voulez-Vous,"[{'id': '4bf58dd8d48988d1d8941735', 'name': 'G...",v-1611409855,False,18 rue du Temple,48.858531,2.353510,"[{'label': 'display', 'lat': 48.85853064543159...",253,75004,FR,Paris,Île-de-France,France,"[18 rue du Temple, 75004 Paris, France]",NaN,NaN
3,4adcda08f964a520803321e3,Café Rive Droite,"[{'id': '4bf58dd8d48988d120941735', 'name': 'K...",v-1611409855,False,2 rue Berger,48.860847,2.348837,"[{'label': 'display', 'lat': 48.86084691113991...",500,75001,FR,Paris,Île-de-France,France,"[2 rue Berger, 75001 Paris, France]",NaN,NaN
4,59a1833193bd63511bd5629d,Terres de Café,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1611409855,False,14 rue Rambuteau,48.860580,2.355482,"[{'label': 'display', 'lat': 48.86057998031148...",523,75003,FR,Paris,Île-de-France,France,"[14 rue Rambuteau, 75003 Paris, France]",NaN,NaN


In [12]:
# assign relevant part of JSON to venues
vienna_venues = vienna_results['response']['venues']

# tranform venues into a dataframe
vienna_dataframe = json_normalize(vienna_venues)
vienna_dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood,location.crossStreet
0,4b963f8df964a52051c234e3,Café de l'Europe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,Graben 31,48.208225,16.371121,"[{'label': 'display', 'lat': 48.2082251445176,...",103,1010,AT,Wien,Wien,Österreich,"[Graben 31, 1010 Wien, Österreich]",NaN,NaN,NaN
1,4c25ebb2f7ced13ac659256d,Café Weinwurm,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,Stephansplatz 11,48.208446,16.372111,"[{'label': 'display', 'lat': 48.2084461064793,...",30,1010,AT,Wien,Wien,Österreich,"[Stephansplatz 11, 1010 Wien, Österreich]",NaN,NaN,NaN
2,4b058890f964a520a7cd22e3,Café-Restaurant Diglas,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1611409855,False,Wollzeile 10,48.208743,16.375210,"[{'label': 'display', 'lat': 48.20874306397518...",205,1010,AT,Wien,Wien,Österreich,"[Wollzeile 10, 1010 Wien, Österreich]",NaN,NaN,NaN
3,4b939600f964a520424934e3,Aida Café,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,Singerstr. 1,48.207992,16.371880,"[{'label': 'display', 'lat': 48.20799192627778...",61,1010,AT,Wien,Wien,Österreich,"[Singerstr. 1, 1010 Wien, Österreich]",76924309,NaN,NaN
4,4b064b09f964a5206dea22e3,Café Sacher,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611409855,False,Philharmonikerstr. 4,48.203857,16.370144,"[{'label': 'display', 'lat': 48.203857, 'lng':...",530,1010,AT,Wien,Wien,Österreich,"[Philharmonikerstr. 4, 1010 Wien, Österreich]",NaN,Innere Stadt,NaN


In [13]:
# keep only columns that include venue name, and anything that is associated with location
london_filtered_columns = ['name', 'categories'] + [col for col in london_dataframe.columns if col.startswith('location.')] + ['id']
london_dataframe_filtered = london_dataframe.loc[:, london_filtered_columns]

paris_filtered_columns = ['name', 'categories'] + [col for col in paris_dataframe.columns if col.startswith('location.')] + ['id']
paris_dataframe_filtered = paris_dataframe.loc[:, paris_filtered_columns]

vienna_filtered_columns = ['name', 'categories'] + [col for col in vienna_dataframe.columns if col.startswith('location.')] + ['id']
vienna_dataframe_filtered = vienna_dataframe.loc[:, vienna_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
london_dataframe_filtered['categories'] = london_dataframe_filtered.apply(get_category_type, axis=1)
paris_dataframe_filtered['categories'] = paris_dataframe_filtered.apply(get_category_type, axis=1)
vienna_dataframe_filtered['categories'] = vienna_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
london_dataframe_filtered.columns = [column.split('.')[-1] for column in london_dataframe_filtered.columns]
paris_dataframe_filtered.columns = [column.split('.')[-1] for column in paris_dataframe_filtered.columns]
vienna_dataframe_filtered.columns = [column.split('.')[-1] for column in vienna_dataframe_filtered.columns]

london_dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Café In The Crypt,Café,St. Martin-in-the-Fields,51.508962,-0.126729,"[{'label': 'display', 'lat': 51.50896230356763...",193,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[St. Martin-in-the-Fields, London, Greater Lon...",NaN,NaN,4b228bfef964a520c34824e3
1,Cafe On The Square,Café,Trafalgar Sq,51.508304,-0.127786,"[{'label': 'display', 'lat': 51.50830388767846...",109,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,4af4407cf964a520e3f021e3
2,The National Café,Café,Trafalgar Sq,51.508803,-0.127544,"[{'label': 'display', 'lat': 51.50880255694977...",164,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq (St. Martin's Pl), London, Great...",St. Martin's Pl,NaN,4ba3cdf5f964a520606138e3
3,Café Fiori,Italian Restaurant,43 Cranbourn St,51.511280,-0.128596,"[{'label': 'display', 'lat': 51.51128033777337...",445,WC2H 7AN,GB,London,Greater London,United Kingdom,"[43 Cranbourn St, London, Greater London, WC2H...",NaN,NaN,4db549dccda1c57c826ad233
4,Portrait Café,Café,NaN,51.509324,-0.127369,"[{'label': 'display', 'lat': 51.50932448760624...",223,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,55a7786b498ecd02e85c3b26
5,MD's Sandwiches & Café,Sandwich Place,38 William IV St.,51.509486,-0.126621,"[{'label': 'display', 'lat': 51.50948579920965...",251,WC2N 4DD,GB,London,Greater London,United Kingdom,"[38 William IV St., London, Greater London, WC...",NaN,NaN,4d41a68a1bd2a143f7cfff7c
6,ICA Bar & Cafe,Bar,The Mall,51.506116,-0.130709,"[{'label': 'display', 'lat': 51.50611560543741...",251,S W1A,GB,London,Greater London,United Kingdom,"[The Mall, London, Greater London, S W1A, Unit...",NaN,NaN,508be0d9e4b0ca23f5de8ccf
7,Internet Cafe,Internet Cafe,11 Charing Cross Road,51.510002,-0.128335,"[{'label': 'display', 'lat': 51.51000241722489...",302,NaN,GB,London,Greater London,United Kingdom,"[11 Charing Cross Road, London, Greater London...",NaN,NaN,4b4227ddf964a520c8cd25e3
8,Cafe Theatre,Café,St Martin's Court,51.511081,-0.127500,"[{'label': 'display', 'lat': 51.51108107138455...",418,WC2N 4AL,GB,Leicester Square,Greater London,United Kingdom,"[St Martin's Court, Leicester Square, Greater ...",NaN,NaN,4b6c2ea0f964a52010282ce3
9,Café Royal Hotel,Hotel,68 Regent St,51.510210,-0.136324,"[{'label': 'display', 'lat': 51.51020994158144...",681,W1B 4DY,GB,London,Greater London,United Kingdom,"[68 Regent St, London, Greater London, W1B 4DY...",NaN,NaN,50c070d7e4b06f3d184f48ce


In [14]:
paris_dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Café L’Étincelle,Café,42 bis rue de Rivoli,48.856623,2.355777,"[{'label': 'display', 'lat': 48.85662349708642...",316,75004,FR,Paris,Île-de-France,France,"[42 bis rue de Rivoli, 75004 Paris, France]",NaN,4afac61bf964a5209c1822e3
1,Café Clémentine,Café,4 rue de la Coutellerie,48.857211,2.350346,"[{'label': 'display', 'lat': 48.85721121243165...",99,75004,FR,Paris,Île-de-France,France,"[4 rue de la Coutellerie, 75004 Paris, France]",NaN,51c5c23d498e3a8ed20914ba
2,Café Voulez-Vous,Gay Bar,18 rue du Temple,48.858531,2.353510,"[{'label': 'display', 'lat': 48.85853064543159...",253,75004,FR,Paris,Île-de-France,France,"[18 rue du Temple, 75004 Paris, France]",NaN,4bb5dd68941ad13a6c941ee3
3,Café Rive Droite,Karaoke Bar,2 rue Berger,48.860847,2.348837,"[{'label': 'display', 'lat': 48.86084691113991...",500,75001,FR,Paris,Île-de-France,France,"[2 rue Berger, 75001 Paris, France]",NaN,4adcda08f964a520803321e3
4,Terres de Café,Coffee Shop,14 rue Rambuteau,48.860580,2.355482,"[{'label': 'display', 'lat': 48.86057998031148...",523,75003,FR,Paris,Île-de-France,France,"[14 rue Rambuteau, 75003 Paris, France]",NaN,59a1833193bd63511bd5629d
5,Café du Châtelet,Café,8 rue Saint-Denis,48.858547,2.347379,"[{'label': 'display', 'lat': 48.8585470820035,...",363,75001,FR,Paris,Île-de-France,France,"[8 rue Saint-Denis, 75001 Paris, France]",NaN,4bbcc5db8ec3d13ac3351b28
6,Café Comptoir,Café,Rue Du Temple,48.858059,2.352887,"[{'label': 'display', 'lat': 48.85805893017868...",184,NaN,FR,Paris,Île-de-France,France,"[Rue Du Temple, Paris, France]",NaN,50959204e4b0e52fa3dd4006
7,Shakespeare & Company Café,Coffee Shop,37 rue de la Bûcherie,48.852527,2.347244,"[{'label': 'display', 'lat': 48.85252686799679...",557,75005,FR,Paris,Île-de-France,France,"[37 rue de la Bûcherie, 75005 Paris, France]",NaN,562a40c3498e425bd63baa85
8,Le Café Livres,Bar,10 rue Saint-Martin,48.857825,2.349288,"[{'label': 'display', 'lat': 48.85782470178464...",202,75004,FR,Paris,Île-de-France,France,"[10 rue Saint-Martin, 75004 Paris, France]",NaN,4b841862f964a520752031e3
9,Bagel Baget Cafe,Bagel Shop,91 rue de la Verrerie,48.858841,2.350697,"[{'label': 'display', 'lat': 48.858841, 'lng':...",245,75004,FR,Paris,Île-de-France,France,"[91 rue de la Verrerie, 75004 Paris, France]",NaN,53d13101498e918e659f88a2


In [17]:
vienna_dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Café de l'Europe,Café,Graben 31,48.208225,16.371121,"[{'label': 'display', 'lat': 48.2082251445176,...",103,1010,AT,Wien,Wien,Österreich,"[Graben 31, 1010 Wien, Österreich]",NaN,NaN,4b963f8df964a52051c234e3
1,Café Weinwurm,Café,Stephansplatz 11,48.208446,16.372111,"[{'label': 'display', 'lat': 48.2084461064793,...",30,1010,AT,Wien,Wien,Österreich,"[Stephansplatz 11, 1010 Wien, Österreich]",NaN,NaN,4c25ebb2f7ced13ac659256d
2,Café-Restaurant Diglas,Restaurant,Wollzeile 10,48.208743,16.375210,"[{'label': 'display', 'lat': 48.20874306397518...",205,1010,AT,Wien,Wien,Österreich,"[Wollzeile 10, 1010 Wien, Österreich]",NaN,NaN,4b058890f964a520a7cd22e3
3,Aida Café,Café,Singerstr. 1,48.207992,16.371880,"[{'label': 'display', 'lat': 48.20799192627778...",61,1010,AT,Wien,Wien,Österreich,"[Singerstr. 1, 1010 Wien, Österreich]",NaN,NaN,4b939600f964a520424934e3
4,Café Sacher,Café,Philharmonikerstr. 4,48.203857,16.370144,"[{'label': 'display', 'lat': 48.203857, 'lng':...",530,1010,AT,Wien,Wien,Österreich,"[Philharmonikerstr. 4, 1010 Wien, Österreich]",Innere Stadt,NaN,4b064b09f964a5206dea22e3
5,Café Hawelka,Café,Dorotheergasse 6,48.207844,16.369849,"[{'label': 'display', 'lat': 48.2078439788914,...",204,1010,AT,Wien,Wien,Österreich,"[Dorotheergasse 6, 1010 Wien, Österreich]",Innere Stadt,NaN,4b058890f964a520aecd22e3
6,Chattanooga Cafe & Grill,Café,Graben 29A,48.208440,16.370520,"[{'label': 'display', 'lat': 48.20844, 'lng': ...",147,1010,AT,Wien,Wien,Österreich,"[Graben 29A, 1010 Wien, Österreich]",NaN,NaN,4bf1868a24f19c743269f983
7,Hard Rock Cafe,American Restaurant,Rotenturmstraße 25,48.211317,16.375482,"[{'label': 'display', 'lat': 48.21131735081294...",397,1010,AT,Wien,Wien,Österreich,"[Rotenturmstraße 25, 1010 Wien, Österreich]",NaN,NaN,534aa8ce498ecf32f55f8d1c
8,Café Bar Bloom,Café,Rotenturmstraße 15,48.210227,16.374049,"[{'label': 'display', 'lat': 48.21022685504435...",237,1010,AT,Wien,Wien,Österreich,"[Rotenturmstraße 15 (Lichtensteg), 1010 Wien, ...",Kärntner Viertel,Lichtensteg,50b0aa2ee4b0a74d35142017
9,Café Restaurant Bräunerhof,Café,Stallburggasse 2,48.207189,16.368534,"[{'label': 'display', 'lat': 48.20718918689899...",321,1010,AT,Wien,Wien,Österreich,"[Stallburggasse 2, 1010 Wien, Österreich]",NaN,NaN,4b058890f964a520a6cd22e3


In [26]:
print("The ammount of cafes are:\nLondon: ",london_dataframe_filtered.size,"\n Paris: ",paris_dataframe_filtered.size,"\nVienna: ",vienna_dataframe_filtered.size)

The ammount of cafes are:
London:  480 
 Paris:  450 
Vienna:  480


As the number of cafes in London and Vienna are equal, I will choose the city that has a lower distance average, so that our client spends less time commuting

In [29]:
print("Average distance for: \n London: ",london_dataframe_filtered[['distance']].mean(axis=0),"\n Vienna: ", vienna_dataframe_filtered[['distance']].mean(axis=0))

Average distance for: 
 London:  distance    495.166667
dtype: float64 
 Vienna:  distance    555.3
dtype: float64


In [ ]:
London has the higher ammount of cafes and 